![NASAASO](Images/ASOInc_QuandaryPeak_web.jpg)

# Retreiving, Processing, and Analyzing NASA Airborne Snow Observatory (ASO) SWE data product

In this exercise, the user will programatically retreive, process, and spatio temporally analyze swe from NASA ASO missions


# 1. Create Map for Watershed for USGS Station ID
The following code uses the pynhd and folium packages to create an interactive map of a watershed from a USGS gauge ID.

In our exercise, we are tasked with identifying all SNOTEL sites upstream of Hetch Hetchy Reservoir on the Tuolumne River. The user can search for "USGS streamflow Tuolumne River" and serveral locations will pop up. Site [11274790](https://waterdata.usgs.gov/monitoring-location/11274790/#dataTypeId=continuous-00065-0&period=P7D&showMedian=false) is the site of interest for this assessment 

In [ ]:
from pynhd import NLDI, WaterData, NHDPlusHR, GeoConnex
import geopandas as gpd
import pandas as pd
from supporting_scripts import ASOget, dataprocessing, mapping, SSWEET, get_Elevation
from shapely.geometry import box, Polygon
import os
import datetime
import matplotlib.pyplot as plt
import numpy as np
import zipfile
import os
import warnings
warnings.filterwarnings("ignore")


Define the watershed outlet using NWIS site id. Create a map object that we'll add layers to.

In [ ]:
nldi = NLDI()
usgs_gage_id = "11274790" # NWIS id for Tuolumne river at the mouth of Hetch Hetchy Reservoir

Collect watershed and reach vectors using the pynhd module.

In [ ]:
nldi = NLDI()

#Getting basin geometry
print('Collecting basins...', end='')
basin = nldi.get_basins(usgs_gage_id)
if not os.path.exists('files'):
    os.makedirs('files')
basin.to_file("files/TuolumneRiverBasin.shp")
print('done')

site_feature = nldi.getfeature_byid("nwissite", f"USGS-{usgs_gage_id}")
upstream_network = nldi.navigate_byid(
    "nwissite", f"USGS-{usgs_gage_id}", "upstreamMain", "flowlines", distance=9999
)

Create and interactive map to display the watershed.

In [ ]:
# create map
mapping.basin_mapping(basin, site_feature)

![Basin](Images/basin.png)

# 2. Retrieve NASA ASO SWE data product for basin of interest

note*, new users will have to [create a .netrc file](https://earthaccess.readthedocs.io/en/latest/howto/authenticate/):

import earthaccess

earthaccess.login(persist=True)

*note, this script can time some time to process(~10 minutes), is prone to loosing html connection, and will grab any NASA ASO Image within the bounding box of your basin of interest, making the reproduction of this code tedious.

In the spirit of the module, ee have QA/QC'd these for you, which we zip and include for you in this repository.

In [ ]:
# import earthaccess https://earthaccess.readthedocs.io/en/latest/howto/authenticate/
# earthaccess.login(persist=True)

# Inputs for fetching ASO data for a region
short_name = 'ASO_50M_SWE'
version = '1'
time_start = '2013-04-02T00:00:00Z'
time_end = '2019-07-19T23:59:59Z'
output_res = 1000 #desired spatial resoultion in meters (m)
directory = "Raw_ASO_Data"
basinname = 'Tuolumne'

#Get ASO data
folder_name = f"files/ASO/{basinname}/{directory}"
data_tool = ASOget.ASODownload(short_name, version)
b_box = data_tool.BoundingBox(basin)  
url_list = data_tool.cmr_search(time_start, time_end, basin, b_box)
data_tool.cmr_download(folder_name, basinname)

#Convert ASO tifs to parquet
data_processor = ASOget.ASODataProcessing()
data_processor.convert_tiff_to_parquet_multiprocess(directory, output_res, basinname) 

Below is the code to unzip the preprosed ASO images and continue the hydrolearn activity

In [ ]:
output_res = 1000 #desired spatial resoultion in meters (m)
basinname = 'Tuolumne'

# Path to the zip file
zip_file_path = f"files/ASO/{basinname}/{output_res}M_SWE_parquet.zip"


# Path to the directory where you want to extract the files
extract_to_path = f"files/ASO/{basinname}/{output_res}M_SWE_parquet/"

# Ensure the extraction directory exists
os.makedirs(extract_to_path, exist_ok=True)

# Unzip the file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to_path)

print(f"Files have been extracted to {extract_to_path}")

In [ ]:
cmap = 'seismic' # use seismic for error, viridis or blues for preds/obs
var =  'swe_in' #'error'
savfig = True
variant = 'World_Imagery'
markersize = 60
swethres = 0.1
plttitle = 'N/A'

#Get all file names for ASO images in Tuolumne river basin
files = [f for f in os.listdir(f"files/ASO/{basinname}/{output_res}M_SWE_parquet/") if os.path.isfile(os.path.join(f"files/ASO/{basinname}/{output_res}M_SWE_parquet/", f))]
files = [files[0]]

SSWEET.SpatialAnalysis(files, basinname, output_res, markersize, cmap, var,variant,swethres,plttitle, pltfig = True, savfig=False)

# 3. Generate spatiotemporal average SWE values

Apply methods to determine the patial mean/median SWE for the Tuolumne watershed above hetch hetchy reservoir for a specific time of year (e.g., Peak SWE)

In [ ]:
basinname = 'Tuolumne'
output_res = 1000 #desired spatial resoultion in meters (m)
 # get files of interest
begdate = 325 #march 25th
enddate = 407 #april 7th
filename = f"{basinname}_median_SWE_04-01.parquet"
decround = 2

MedianSWE_df = dataprocessing.Spatial_median_SWE_df(output_res, basinname, begdate, enddate, filename,decround,  save = True)
MedianSWE_df

In [ ]:
#Plot median SWE
basinname = 'Tuolumne'
output_res = 1000 #desired spatial resoultion in meters (m)
 # get files of interest
filename = [f"{basinname}_median_SWE_04-01.parquet"]
cmap = 'viridis' # use seismic for error, viridis or blues for preds/obs
var =  'median_SWE_in' #'error'
savfig = True
variant = 'World_Imagery'
markersize = 60
swethres = 0.1
plttitle = 'Median SWE in Tuolumne River Basin on April 1st'

SSWEET.SpatialAnalysis(filename, basinname, output_res, markersize, cmap, var,variant,swethres,plttitle, pltfig = True, savfig=False)

# 4. Spatially compare a year of interest to the historical swe value


In [ ]:
basinname = 'Tuolumne'
output_res = 1000 #desired spatial resoultion in meters (m)
medianSWEfile = f"{basinname}_median_SWE_04-01.parquet"
date = '20190324' #date of interest
WYSWEfile = f"ASO_{output_res}M_SWE_{date}.parquet"
swedifffilename = f"{basinname}_SWEDiff_{date}.parquet"
decround = 2

df = dataprocessing.SWE_diff(basinname, output_res, medianSWEfile, WYSWEfile,decround,swedifffilename, save=True)


In [ ]:
#Plot the SWE difference
basinname = 'Tuolumne'
output_res = 1000 #desired spatial resoultion in meters (m)
 # get files of interest
filename = [swedifffilename]
cmap = 'seismic_r' # use seismic for error, viridis or blues for preds/obs
var =  'SWE_diff_in' #'error'
savfig = True
variant = 'World_Imagery'
markersize = 60
swethres = 0.1
plttitle = 'SWE Difference to Median in Tuolumne River Basin on April 1st'

SWEdiff = SSWEET.SpatialAnalysis(filename, basinname, output_res, markersize, cmap, var,variant,swethres,plttitle, pltfig = True, savfig=False)

In [ ]:
#Plot the SWE difference
basinname = 'Tuolumne'
output_res = 1000 #desired spatial resoultion in meters (m)
 # get files of interest
filename = [swedifffilename]
cmap = 'seismic_r' # use seismic for error, viridis or blues for preds/obs
var =  'SWE_perc_norm' #SWE_diff_in' #'error',
savfig = True
variant = 'World_Imagery'
markersize = 60
swethres = 0.1
plttitle = 'SWE Percentage Difference to Median in Tuolumne River Basin on April 1st'

SWEdiff = SSWEET.SpatialAnalysis(filename, basinname, output_res, markersize, cmap, var,variant,swethres,plttitle, pltfig = True, savfig=False)

# 5. Investigate the differences between median and current basin snow conditions

In [ ]:
#get elevation data for each pixel in the basin, this may take a few moments
SWEdiff_elev = get_Elevation.extract_terrain_data_threaded(SWEdiff,basinname, output_res)
SWEdiff_elev.head()

In [ ]:
incols = ['median_SWE_in', "swe_in"]
#outcols =['Volume_Difference', 'Median_Volume', 'Observed_Volume']
outcols =['median_SWE_in', "swe_in"]
ncol = 3
output_res = 1000
region = 'Sierra Nevada'

Title = f'Volumetric Frozen Water Content at Low, Mid, and High Elevation Bands \n {basinname} River Basin, {region} {date}'
save = True
figname = f"Figures/ASO/{basinname}/{output_res}M/{basinname}_ElevationVol_{date}.png"

Depthdf = SSWEET.barplot(SWEdiff_elev, incols, outcols, output_res, ncol, Title, save, figname)

In [ ]:
incols = ['median_SWE_in', "swe_in"]
outcols =['Volume_Difference', 'Median_Volume', 'Observed_Volume']
region = 'Sierra Nevada'
ncol = 3
output_res = 1000
Title = f'Volumetric Frozen Water Content at Low, Mid, and High Elevation Bands \n {basinname} River Basin, {region} {date}'
save = True
figname = f"Figures/ASO/{basinname}/{output_res}M/{basinname}_ElevationVol_{date}.png"

Vdf = SSWEET.barplot(SWEdiff_elev, incols, outcols, output_res, ncol, Title, save, figname)

In [ ]:

incols = ['SWE_perc_norm']
outcols =['SWE_perc_norm']
region = 'Sierra Nevada'
ncol = 3
output_res = 1000
Title = f'Volumetric Frozen Water Content at Low, Mid, and High Elevation Bands \n {basinname} River Basin, {region} {date}'
save = True
figname = f"Figures/ASO/{basinname}/{output_res}M/{basinname}_ElevationPerc_{date}.png"

Percdf = SSWEET.barplot(SWEdiff_elev, incols, outcols, output_res, ncol, Title, save, figname)